In [1]:
import sys
sys.path.append('../src')
import os

import importlib
import visualize
importlib.reload(visualize)
import matplotlib.pyplot as plt
import pandas as pd

In [17]:
COLS = ["Давление в системе", "Температура масла в баке"]

In [6]:
report_alerts_xlsx = pd.read_excel('/home/jupyter/datasphere/project/diplom/reports/thr=0.05_trend_level_2022_3q_2023_4q_alerts.xlsx')
report_alerts_xlsx = report_alerts_xlsx.set_index('Дата и время начала срабатывания предупреждения модели')
report_alerts_xlsx.head(1)

,Дата и время окончания срабатывания предупреждения модели,НШС узла в течение 10 дней,Время НШС,Признак: спрогнозировано ли НШС,"Показатели, сработавшие в модели как определяющие предупреждение (список)",Срок от предупреждения до начала НШС
Дата и время начала срабатывания предупреждения модели,,,,,,
2022-07-09 04:26,2022-07-09 06:26,NaN,NaN,False,Температура масла в баке_trend\nУровень масла ...,NaN


In [15]:
fact_predict_total = pd.read_pickle('/home/jupyter/datasphere/project/diplom/reports/thr=0.05_trend_level_2022_3q_2023_4q_fact_predict_total.pickle')
fact_predict_total.reset_index(inplace=True)

test_df = fact_predict_total[['Time', 'Давление в системе_test', 'Температура масла в баке_test']]
test_df = test_df.rename(columns=lambda x: x.replace('_test', ''))
test_df.reset_index(inplace=True)
test_df['Time'] = pd.to_datetime(test_df['Time'])
test_df.set_index('Time', inplace = True)

predict_df = fact_predict_total[['Time','Давление в системе_pred', 'Температура масла в баке_pred']]
predict_df = predict_df.rename(columns=lambda x: x.replace('_pred', ''))
predict_df.reset_index(inplace=True)
predict_df['Time'] = pd.to_datetime(predict_df['Time'])
predict_df.set_index('Time', inplace = True)

anomaly_vals_total = pd.read_pickle('/home/jupyter/datasphere/project/diplom/reports/thr=0.05_trend_level_2022_3q_2023_4q_anomaly_vals_total.pickle')
anomaly_vals_total.reset_index(inplace=True)
anomaly_vals_total['Time'] = pd.to_datetime(anomaly_vals_total['Time'])
anomaly_vals_total.set_index('Time', inplace = True)

In [18]:
for index, row in report_alerts_xlsx.iterrows():
    timestamp = pd.Timestamp(index) - pd.Timedelta(hours = 30)
    alert_end = pd.Timestamp(row['Дата и время окончания срабатывания предупреждения модели']) + pd.Timedelta(hours = 30)
    visualize.plot_multiple_cols(COLS, test_df, predict_df, anomaly_vals_total, 
                       alert_start=timestamp, alert_end=alert_end,
                       thresh = 0.05, save_folder = 'trend_level_0.05_buf')